In [1]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
import re
from time import sleep as time_sleep
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED

In [2]:
url = 'https://lenta.com/catalog/'
host = '/'.join(url.split('/', maxsplit=3)[:3])

In [3]:
class initial_driver(NamedTuple):
    browserWidth: int
    browserHeight: int
    browserLeft: int
    browserTop: int
    proxy: str
    start_url: str
    
test = initial_driver(1280, 1300, -2560, 0, '45.81.76.252:8000', url)

In [5]:
def _start_driver(initial:initial_driver) -> uc.Chrome:
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument(f'--proxy-server={initial.proxy}')
    chrome_options.add_argument(f"--window-position={initial.browserLeft},{initial.browserTop}")
    chrome_options.add_argument(f"--window-size={initial.browserWidth},{initial.browserHeight}")
    driver = uc.Chrome(options=chrome_options)
    driver.get(initial.start_url)
    time_sleep(2)
    return driver




In [6]:
# Сохранение ссылок на категории продуктов
def _save_link_to_category(driver: uc.Chrome):
    soup = BS(driver.page_source, 'html.parser')

    table = soup.find('div', class_='catalog-groups-grid catalog-groups-grid--small-paddings')
    link_dict = {}
    for item in table.find_all('a'):
        name = item.get('ga-event-label')
        href = host + item.get('href')
        link_dict[name] = href
    link_dict

    # Сохраняем в JSON файл
    with open('data/category_link_dict.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(link_dict, indent=4, ensure_ascii=False))

In [8]:
def select_sity(driver: uc.Chrome):
    # Заставляем Браузер выбрать на сайте интерисующий нас магазин
    
    # Функция ожидания, позволяет Браузеру дождаться загрузки элементов
    wait = WebDriverWait(
        driver,
        timeout=40,
        poll_frequency=1,
        ignored_exceptions=[TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException])

    # Тут происходит список действий необходимый для выбора магазина
    try:
        element = wait.until(lambda d: d.find_element(by=By.CLASS_NAME, value='address-container__adress-icon-container'))
    except Exception as ex:
        print('!!!select_sity 52',ex)
    element.click()
    
    try:
        element = wait.until(lambda d: d.find_element(by=By.CLASS_NAME, value='store-picker-city__label'))
    except Exception as ex:
        print('!!!select_sity 58',ex)
    element.click()
    
    try:
        elements = wait.until(lambda d: d.find_elements(by=By.CSS_SELECTOR, value='span.cities-list-item__name'))
    except Exception as ex:
        print('!!!select_sity 64',ex)
    for el in elements:
        if 'Липецк' in el.text:
            el.click()
            break
    try:
        elements = wait.until(lambda d: d.find_elements(by=By.CSS_SELECTOR, value='span.button__inner'))
    except Exception as ex:
        print('!!!select_sity 72',ex)
    for el in elements:
        if 'Магазины' in el.text:
            el.click()
            break
    
    try:
        elements = wait.until(lambda d: d.find_element(by=By.CLASS_NAME, value='simplebar-content').find_elements(by=By.CLASS_NAME, value="stores-list-item__name"))
    except Exception as ex:
        print('!!!select_sity 81',ex)
    for el in elements:
        if 'ул. Катукова, д. 51' in el.text:
            el.click()
            break
    
    try:
        element = driver.find_element(by=By.XPATH, value="//*[contains(text(), 'Выбрать магазин')]")
    except Exception as ex:
        print('!!!select_sity 90',ex)
    element.click()
    time_sleep(5)

In [9]:
def _get_pagination(driver: uc.Chrome) -> int:
    el = driver.find_element(by=By.CLASS_NAME, value='js-catalog-container')
    catalog = json.loads(el.get_attribute('data-catalog-data'))
    pagination = math.ceil(catalog.get('skusCount')/catalog.get('limit'))
    return pagination

In [10]:
def _get_queue_cat(driver: uc.Chrome) -> Queue:
    page_url = driver.current_url
    pagination = _get_pagination(driver)
    my_queue = Queue()
    for key in range(1, pagination+1):
        my_queue.put(f'{page_url}?page={key}')
    return my_queue

In [11]:
def _save_pages(pages_list: list[str], name: str = 'lenta') -> None:
    now = datetime.now()
    formatted_date = now.strftime('%Y-%m-%d_%H-%M')
    with ZipFile(f'data/{formatted_date}-{name}.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
        for index, page in enumerate(pages_list):
            zip_file.writestr(f'{index:05d}.html', page)

In [12]:
with open('data/category_link_dict.json', 'r', encoding='utf-8') as file:
    link_dict = json.loads(file.read())

link_queue = Queue()
for val in link_dict.values():
    link_queue.put(val)

def get_pages(initial_tuple: initial_driver) -> list:
    output_list = []
    
    driver = _start_driver(initial_tuple)
    select_sity(driver)
    count = 0
    while True:
        url = link_queue.get()
        driver.get(url)
        time_sleep(2)
        cat_queue = _get_queue_cat(driver)
        
        while True:
            url = cat_queue.get()
            driver.get(url)
            time_sleep(1)
            output_list.append(driver.page_source)
            
            if cat_queue.empty():
                break
        
        count += 1
        if count == 2:
            break
        if link_queue.empty():
            break
        
    driver.quit()
    return output_list
page_list = get_pages(test)

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:58295
from session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.110
Stacktrace:
Backtrace:
	GetHandleVerifier [0x007FA813+48355]
	(No symbol) [0x0078C4B1]
	(No symbol) [0x00695358]
	(No symbol) [0x006B61AC]
	(No symbol) [0x006B06B4]
	(No symbol) [0x006B0491]
	(No symbol) [0x006E0C55]
	(No symbol) [0x006E093C]
	(No symbol) [0x006DA536]
	(No symbol) [0x006B82DC]
	(No symbol) [0x006B93DD]
	GetHandleVerifier [0x00A5AABD+2539405]
	GetHandleVerifier [0x00A9A78F+2800735]
	GetHandleVerifier [0x00A9456C+2775612]
	GetHandleVerifier [0x008851E0+616112]
	(No symbol) [0x00795F8C]
	(No symbol) [0x00792328]
	(No symbol) [0x0079240B]
	(No symbol) [0x00784FF7]
	BaseThreadInitThunk [0x751F7D59+25]
	RtlInitializeExceptionChain [0x76F4B79B+107]
	RtlClearBits [0x76F4B71F+191]


In [187]:
_save_pages(page_list)